## 4. 模型解码/预测

In [25]:
import os
import pathlib
import pickle
import random
from functools import reduce

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import librosa
import numpy as np
from tqdm import tqdm
from scipy.fftpack import dct

import tensorflow as tf

keras = tf.keras
ctc_decode, get_value, load_model = (
    keras.backend.ctc_decode,
    keras.backend.get_value,
    keras.models.load_model,
)

# 音频/语音标注文件路径
DS_PATH = "data/"
# 模型文件路径
FILES_PATH = "output/"

# 音频文件路径
data_path = sorted([str(p) for p in pathlib.Path(DS_PATH).glob("*.wav")])
# 语音标注文件路径
label_path = sorted([str(p) for p in pathlib.Path(DS_PATH).glob("*.trn")])


### 0. 加载模型与数据

In [26]:
def load_dataset_mfcc(
    file_path,
    sr=16000,
    n_mfcc=20,
    n_fft=512,
    min_db=23,
    emphasis=0.97,
    hop_length=0.01,
    win_length=0.025,
    lifter=22,
    is_emphasis=False,
):
    """
    加载并提取音频特征, 返回经过预处理的音频mfcc数组
    :param file_path:    list 音频文件路径
    :param sr:           int 音频采样率
    :param n_mfcc:       int mfcc特征维度
    :param n_fft:        int stft计算点数
    :param min_db:       float 删除静音片段的最小分贝数
    :param emphasis:     float 预加重系数
    :param hop_length:   float 分帧间隔长度
    :param win_length:   float 分窗长度
    :param lifter:       int 对倒谱应用系数提升, 数值为正数
    :param is_emphasis:  bool 是否进行预加重
    :return:             np.ndarray 包含所有音频文件的mfcc特征二维数组 (frames, n_mfcc)
    """
    ds = list()
    for path in tqdm(file_path):
        # 读取文件
        y, sr = librosa.load(path=path, sr=sr)

        # 去除音频中所有的空白静默部分
        y_split = librosa.effects.split(y, top_db=min_db)
        y_split = np.array(list(reduce(lambda x, y: np.concatenate((x, y)), [y[x[0] : x[1]] for x in y_split])))

        # 预加重
        if is_emphasis:
            y_split = librosa.effects.preemphasis(y_split, coef=emphasis)

        # 提取Mel频谱
        y_mel = librosa.feature.melspectrogram(
            y=y_split, sr=sr, n_fft=n_fft, hop_length=int(sr * hop_length), win_length=int(sr * win_length)
        )

        # 对分贝频谱应用DCT得到MFCC特征
        y_db = librosa.power_to_db(y_mel)
        y_mfcc = dct(y_db, axis=-2, type=2, norm="ortho")[..., :n_mfcc, :]

        # 对MFCC应用提升系数, 可以提高高频部分的分辨率
        if lifter > 0:
            n_lifter = np.sin(np.pi * np.arange(1, 1 + n_mfcc, dtype=y_mfcc.dtype) / lifter)
            n_lifter = librosa.util.expand_to(n_lifter, ndim=y_db.ndim, axes=-2)
            y_mfcc *= 1 + (lifter / 2) * n_lifter

        # 保存数据 (转置是为了与后续的模型输入层维度匹配)
        ds.append(y_mfcc.transpose())

    return ds

In [27]:
# 读取音频模型均值, 标准差
with open(FILES_PATH + "dataset/data_mfcc.pkl", "rb") as file:
    _, mfcc_mean, mfcc_std = pickle.load(file)
    del _

# 读取词库
with open(FILES_PATH + "dataset/words_vec.pkl", "rb") as file:
    char2id, id2char = pickle.load(file)

In [28]:
# 导入保存的模型
bigru_model = load_model(FILES_PATH + 'models/bigru.h5')
# wavenet_model = load_model(FILES_PATH + 'wavenet.h5')

### 1. 构建CTC解码模型
CTC编码后的模型需要解码后才能输出对应标签的概率值  
使用tf.keras.backend.ctc_decode进行解码  

y_pred	        包含预测的张量或 softmax 的输出 (samples, time_steps, num_categories)  
input_length	包含预测结果中每个批次序列长度的张量 (samples, 1)  
greedy	        如果true, 则执行更快的最佳路径搜索  
beam_width	    如果greedy是false, 波束搜索解码器将与此宽度的波束一起使用  
top_paths	    如果greedy是false, 将返回多少条最可能的路径  

In [29]:
def ctc_decode_model(pred_audio, dict_list, model):
    """
    使用模型对音频解码预测其内容
    :param pred_audio:  需要预测的音频特征
    :param dict_list:   转换标签所使用的词库
    :param model:       预测使用的模型
    :return:            (str, list) 返回预测的文本及其标签序号
    """
    # 使用模型预测结果 (expand_dims用于增加维度匹配模型输入)
    pred = model.predict(np.expand_dims(pred_audio, axis=0))
    # 音频帧数
    input_length = np.array((pred_audio.shape[1],))

    # 解码
    decode_res = ctc_decode(pred, input_length, greedy=True, beam_width=100, top_paths=1)
    # 获取预测的序列数组, 筛选有效值 (> -1)
    pred_index = get_value(decode_res[0][0])
    pred_index = [item for item in pred_index[0] if item > -1]

    # 使用词库将预测标号转换为文本
    pred_text = ""
    for index in pred_index:
        pred_text += dict_list[index]

    # 返回预测的文本及其标号
    return pred_text, pred_index

In [30]:
# mfcc特征维数
MFCC_VALUE = 13
# FFT计算点数
FFT = 551

# 随机读取读取一条语音
index = random.randint(0, len(data_path) - 1)
sound_mfcc = load_dataset_mfcc([data_path[index]], n_mfcc=MFCC_VALUE, is_emphasis=True, n_fft=FFT)[0]
# 标准化
sound_mfcc = (sound_mfcc - mfcc_mean) / mfcc_std

# 读取语音对应文本
with open(label_path[index], "r") as file:
    sound_text = file.readline().strip()

print(sound_text)
sound_mfcc

100%|██████████| 1/1 [00:00<00:00, 28.69it/s]

卢 泰 愚 因 犯 有 主动 参与 军事 叛乱 和 内乱 罪 谋杀 上司 未遂 罪 及 受贿罪 等 被 判处 无期徒刑


array([[-1.0784433 ,  0.05292991,  0.5758086 , ...,  0.5162519 ,
         0.78842825,  0.31906   ],
       [-1.6075298 ,  0.36116675,  0.8754167 , ...,  0.796005  ,
         1.4950706 , -0.40185764],
       [-1.5233955 ,  0.7228484 ,  1.1778085 , ..., -0.00876884,
         1.2226315 , -1.0876694 ],
       ...,
       [-1.1192462 ,  1.6465904 ,  0.1690056 , ..., -0.594472  ,
        -0.25677702, -0.8000269 ],
       [-1.2128937 ,  1.612051  ,  0.28389165, ..., -0.5397985 ,
        -0.4792143 , -0.729339  ],
       [-1.1986645 ,  1.8318487 ,  0.651445  , ..., -0.00754732,
        -0.12298144, -1.0198172 ]], dtype=float32)

In [31]:
bigru_text, bigru_textid = ctc_decode_model(sound_mfcc, id2char, bigru_model)

print("原文:", sound_text.replace(" ", ""))
print("识别:", bigru_text)
print("标号:", bigru_textid)

1/1 [==============================] - 1s 1s/step
原文: 卢泰愚因犯有主动参与军事叛乱和内乱罪谋杀上司未遂罪及受贿罪等被判处无期徒刑
识别: 卢泰
标号: [2066, 1442]


In [32]:
# wavenet_text, wavenet_textid = ctc_decode_model(sound_mfcc, id2char, wavenet_model)

# print("原文:", sound_text.replace(" ", ""))
# print("识别:", wavenet_text)
# print("标号:", wavenet_textid)